In [1]:
import robot
import map
import solver
import utils
import transforms2D
import scenarios

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

import gtsam
from gtsam.symbol_shorthand import L, X

# unitTest 0: Build Circular Trajectory

In [16]:
r = 1
m = 20 #steps
dtheta = np.pi * 2 / m
x0, y0, theta0 = 0, 0, 0 #circle trajectory parameters
pose_ii = []
for ii in range(m):
    theta = theta0 + ii*dtheta #circle parameter
    alpha = np.pi/2 +theta #ego tangent vector angle relative to x vector
    t_w_w2ii = np.array([[r * np.cos(theta)- x0],
                   [r * np.sin(theta)- y0]])
    
    pose = gtsam.Pose2(t_w_w2ii[0],t_w_w2ii[1],alpha)
    pose_ii.append(pose)


fig , ax = utils.spawnWorld(xrange = (-2*r,2*r), yrange = (-2*r,2*r))
for pose in pose_ii:
    utils.plot_pose(ax ,pose)
    plt.pause(0.1)

dpose = (gtsam.Pose2.between(pose_ii[0],pose_ii[1]))
print(f"dpose type: {type(dpose)}")
print(f"dpose value {dpose}")
print(f"pose0 + dpose == pose1? {pose_ii[0].compose(dpose).equals(pose_ii[1],tol = 0.001)}")


dpose type: <class 'gtsam.gtsam.Pose2'>
dpose value (0.309017, 0.0489435, 0.314159)

pose0 + dpose == pose1? True


# unitTest 1: Move robot and check odometry measurements

In [4]:
pose0 = gtsam.Pose2(1.0,0.0,np.pi/2)
car = robot.robot(pose = pose0)

#control inputs
r = 1
m = 20
dx = 0.309017 #taken from unitTest0 relative odometrey
dy = 0.0489435
dtheta = 0.314159
gt_odom = [gtsam.Pose2(dx,dy,dtheta)] * 2*m

#init history loggers
hist_GT, hist_DR = car.pose.translation(), car.pose.translation()

#set graphics
fig , ax = utils.spawnWorld(xrange = (-2*r,2*r), yrange = (-2*r,2*r))
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = car.pose
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        print(car.pose)
        
        #dead reckoning integration
        xcurrent_DR = xcurrent_DR.compose(meas_odom)
        
        #log history
        hist_DR = np.vstack([hist_DR,xcurrent_DR.translation()])
        hist_GT = np.vstack([hist_GT,car.pose.translation()])

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(0.1)

(0.951056, 0.309017, 1.88496)

(0.809017, 0.587785, 2.19911)

(0.587785, 0.809017, 2.51327)

(0.309017, 0.951057, 2.82743)

(2.81876e-07, 1, 3.14159)

(-0.309017, 0.951058, -2.82743)

(-0.587785, 0.809018, -2.51328)

(-0.809017, 0.587787, -2.19912)

(-0.951058, 0.309019, -1.88496)

(-1, 2.29097e-06, -1.5708)

(-0.951059, -0.309015, -1.25664)

(-0.80902, -0.587784, -0.942481)

(-0.587789, -0.809016, -0.628322)

(-0.309021, -0.951056, -0.314163)

(-4.66269e-06, -1, -3.98038e-06)

(0.309013, -0.951059, 0.314155)

(0.587781, -0.80902, 0.628314)

(0.809014, -0.58779, 0.942473)

(0.951055, -0.309022, 1.25663)

(1, -5.30718e-06, 1.57079)

(0.951058, 0.309012, 1.88495)

(0.80902, 0.587781, 2.19911)

(0.58779, 0.809014, 2.51327)

(0.309022, 0.951055, 2.82743)

(5.58906e-06, 1, 3.14159)

(-0.309012, 0.951059, -2.82744)

(-0.587781, 0.809022, -2.51328)

(-0.809014, 0.587791, -2.19912)

(-0.951056, 0.309024, -1.88496)

(-1, 7.59816e-06, -1.5708)

(-0.951061, -0.30901, -1.25665)

(-0.809023, -0.587

In [8]:
car.pose

(0.63662, 0.63662, 1.5708)

# unitTest 2: Build prior map

In [5]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs @ sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [6]:
#------Build worldmap
xrange = (-3,1); yrange = (-2,3)
fig , ax = utils.spawnWorld(xrange, yrange)
np.random.seed(seed=0)

N = 20 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
car.FOV = 2*np.pi
car.range = 1
#control inputs
ds = 0.4 #[m]
dtheta = np.radians(20)
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    while time <= timeFinal:
        meas_odom = car.moveAndMeasureOdometrey(odom)

        meas_lm = car.measureLandmarks(worldMap)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta
        xcurrent_DR[0] += meas_odom.dx * np.cos(xcurrent_DR[2])
        xcurrent_DR[1] += meas_odom.dy * np.sin(xcurrent_DR[2])
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_LM.append(meas_lm)

        #increase time
        time += dt

        #plot
        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        plt.draw()
        plt.pause(dt)

        print(meas_lm)


[<robot.meas_landmark object at 0x7f80d00888b0>, <robot.meas_landmark object at 0x7f80c9abac40>, <robot.meas_landmark object at 0x7f80c9abad00>, <robot.meas_landmark object at 0x7f80c9abad60>]
[<robot.meas_landmark object at 0x7f80c9abae50>, <robot.meas_landmark object at 0x7f80c9abaeb0>, <robot.meas_landmark object at 0x7f80c9aaef70>, <robot.meas_landmark object at 0x7f80d0114430>]
[<robot.meas_landmark object at 0x7f80d010e430>, <robot.meas_landmark object at 0x7f80c9ae0220>, <robot.meas_landmark object at 0x7f80c9abae20>]
[<robot.meas_landmark object at 0x7f80c9ae0040>, <robot.meas_landmark object at 0x7f80c9ad5820>, <robot.meas_landmark object at 0x7f80c9ad5eb0>]
[<robot.meas_landmark object at 0x7f80c9ae0310>, <robot.meas_landmark object at 0x7f80d0096250>]
[<robot.meas_landmark object at 0x7f80c9acf310>, <robot.meas_landmark object at 0x7f80c9acf1f0>]
[<robot.meas_landmark object at 0x7f80c9cb44c0>, <robot.meas_landmark object at 0x7f80c9ac6a30>]
[]
[<robot.meas_landmark object a

# unitTest4: Plot Robot Cone

In [6]:
#------Build worldmap
xrange = (-3,1); yrange = (-2,3)
fig , ax = utils.spawnWorld(xrange, yrange)
np.random.seed(seed=0)

#------Spawn Robot
car = robot.robot()
car.plot(ax)
#control inputs
r = 1
p = 2*np.pi*r
steps = 20 #time steps
dtheta = 2*np.pi/steps
ds = p/steps
odom = (ds,ds,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

with plt.ion():
    while time <= timeFinal:
        car.moveAndMeasureOdometrey(odom)

        #increase time
        time += dt

        car.plot()
        plt.draw()
        plt.pause(dt)

# unitTest5 GTSAM

notes: </br>
examples folder: /home/alon18/LocalInstalls/miniconda3/envs/smallslam/lib/python3.9/site-packages/gtsam/examples </br>
* imuFactorISAM2Example.py - understand difference between graph.add and graph.push_back </br>
  + never mind, they are the same (?) https://gtsam.org/doxygen/a03480.html#a1bd7154e74538b5934fc616bb5ef7281 </br>
* PlanarSLAMExample.py- factor for range and bearing </br>


from VisualISAMExample.py: </br>
      # If this is the first iteration, add a prior on the first pose to set the coordinate frame </br>
        # and a prior on the first landmark to set the scale </br>

In [4]:
car, worldMap, ax, fig, gt_odom = scenarios.scenario1()
backend = solver.solver(ax = ax, X0cov = car.odometry_noise/1000, semantics = worldMap.exportSemantics())

car.odometry_noise = car.odometry_noise
car.rgbd_noise = car.rgbd_noise
car.FOV = np.pi/2

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_solverResults = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 1)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 1)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        # for meas_lm in meas_lms:
        #     backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #add dtheta 
        meas_dxdy = utils.R2(xcurrent_DR[2]) @ np.array([[meas_odom.dx,meas_odom.dy]]).T #find translation in global coordiantes
        xcurrent_DR[0] += meas_dxdy[0] #update
        xcurrent_DR[1] += meas_dxdy[1]
        
        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_solverResults.append(backend.current_estimate)

        #plot
        car.plot()
        ax.set_title([lm.index for lm in meas_lms])
        backend.plot_landmarks(plotIndex = True, plotSemantics = True)
        backend.plot_poses()

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        
        plt.draw()
        plt.pause(0.2)

KeyboardInterrupt: 

# Scenario 2 - 4

scenario2: linear trajectory, robot range = 10, odom_dx = 1, landmarks-yrange = (-2,+2), range = 10 </br>
scenario3: linear trajectory, robot range = 3, odom_dx = 1, landmarks-yrange = (-2,+2), range = 3 </br>
scenario4: linear trajectory, robot range = 20, odom_dx = 1, landmarks-yrange = (-10,10), range = 20 </br>

In [4]:
car, worldMap, axWorld, axError,  fig, gt_odom = scenarios.scenario4()
backend = solver.solver(ax = axWorld, X0cov = car.odometry_noise/1000, semantics = worldMap.exportSemantics())

#init history loggers
hist_GT, hist_DR, hist_ISAM2 = car.pose.copy(), car.pose.copy(), car.pose.copy()
hist_DR_error, hist_ISAM2_error = np.zeros((1,1)), np.zeros((1,1))

#set graphics
graphic_GT_traj, = axWorld.plot([], [],'ko-',markersize = 3)
graphic_DR_traj, = axWorld.plot([], [],'ro-',markersize = 3)
graphic_ISAM2_traj, = axWorld.plot([], [],'bo-',markersize = 3)
graphics_DR_error, = axError.plot([], [], 'ro-', markersize = 3)
graphics_ISAM2_error, = axError.plot([], [], 'bo-', markersize = 3)

fig.legend(handles = [graphic_GT_traj,graphic_DR_traj,graphic_ISAM2_traj], 
        labels = ["ground truth","dead reckoning","ISAM2"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #add dtheta 
        meas_dxdy = utils.rot2(xcurrent_DR[2],np.array([meas_odom.dx,meas_odom.dy])) #find translation in global coordiantes
        xcurrent_DR[0] += meas_dxdy[0] #update
        xcurrent_DR[1] += meas_dxdy[1]

        xcurrent_ISAM2 = utils.pose2ToNumpy(backend.current_estimate.atPose2(X(backend.i)))

        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_ISAM2 = np.vstack([hist_ISAM2,xcurrent_ISAM2])
        hist_DR_error = np.vstack([hist_DR_error,np.linalg.norm(xcurrent_DR[:2]-car.pose[:2])])
        hist_ISAM2_error = np.vstack([hist_ISAM2_error,np.linalg.norm(xcurrent_ISAM2[:2]-car.pose[:2])])

        #plot
        car.plot()
        axWorld.set_title([lm.index for lm in meas_lms])
        #backend.plot_landmarks(plotIndex = False, plotSemantics = False)
        #backend.plot_poses(plotCov = False)

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        graphic_ISAM2_traj.set_data(hist_ISAM2[:,0],hist_ISAM2[:,1])
        graphics_DR_error.set_data(range(len(hist_DR_error)),hist_DR_error)
        graphics_ISAM2_error.set_data(range(len(hist_ISAM2_error)),hist_ISAM2_error)
        axError.relim()
        axError.autoscale_view(True,True,True)

        plt.show()
        plt.pause(0.1)

# Scenario 5

In [2]:
car, worldMap, axWorld, axError,  fig, gt_odom = scenarios.scenario5()
backend = solver.solver(ax = axWorld, X0cov = car.odometry_noise/1000, semantics = worldMap.exportSemantics())

#init history loggers
hist_GT, hist_DR, hist_ISAM2 = car.pose.copy(), car.pose.copy(), car.pose.copy()
hist_DR_error, hist_ISAM2_error = np.zeros((1,1)), np.zeros((1,1))

#set graphics
graphic_GT_traj, = axWorld.plot([], [],'ko-',markersize = 3)
graphic_DR_traj, = axWorld.plot([], [],'ro-',markersize = 3)
graphic_ISAM2_traj, = axWorld.plot([], [],'bo-',markersize = 3)
graphics_DR_error, = axError.plot([], [], 'ro-', markersize = 3)
graphics_ISAM2_error, = axError.plot([], [], 'bo-', markersize = 3)

fig.legend(handles = [graphic_GT_traj,graphic_DR_traj,graphic_ISAM2_traj], 
        labels = ["ground truth","dead reckoning","ISAM2"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
with plt.ion():
    for odom in gt_odom:
        meas_odom = car.moveAndMeasureOdometrey(odom)
        meas_lms = car.measureLandmarks(worldMap)

        backend.i += 1 #increase time index. Must be done before adding measurements as factors
        backend.addOdomMeasurement(meas_odom)
        for meas_lm in meas_lms:
            backend.addlandmarkMeasurement(meas_lm)
        backend.update(N=0)

        #dead reckoning integration
        xcurrent_DR[2] += meas_odom.dtheta #add dtheta 
        meas_dxdy = utils.rot2(xcurrent_DR[2],np.array([meas_odom.dx,meas_odom.dy])) #find translation in global coordiantes
        xcurrent_DR[0] += meas_dxdy[0] #update
        xcurrent_DR[1] += meas_dxdy[1]

        xcurrent_ISAM2 = utils.pose2ToNumpy(backend.current_estimate.atPose2(X(backend.i)))

        #log history
        hist_GT = np.vstack([hist_GT,car.pose])
        hist_DR = np.vstack([hist_DR,xcurrent_DR])
        hist_ISAM2 = np.vstack([hist_ISAM2,xcurrent_ISAM2])
        hist_DR_error = np.vstack([hist_DR_error,np.linalg.norm(xcurrent_DR[:2]-car.pose[:2])])
        hist_ISAM2_error = np.vstack([hist_ISAM2_error,np.linalg.norm(xcurrent_ISAM2[:2]-car.pose[:2])])

        #plot
        car.plot()
        axWorld.set_title([lm.index for lm in meas_lms])
        #backend.plot_landmarks(plotIndex = False, plotSemantics = False)
        #backend.plot_poses(plotCov = False)

        graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
        graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
        graphic_ISAM2_traj.set_data(hist_ISAM2[:,0],hist_ISAM2[:,1])
        graphics_DR_error.set_data(range(len(hist_DR_error)),hist_DR_error)
        graphics_ISAM2_error.set_data(range(len(hist_ISAM2_error)),hist_ISAM2_error)
        axError.relim()
        axError.autoscale_view(True,True,True)

        plt.show()
        plt.pause(0.1)

In [5]:
meas_odom.

[array([10.,  0.,  0.]),
 array([10.,  0.,  0.]),
 array([0.        , 0.        , 1.57079633]),
 array([10.,  0.,  0.]),
 array([10.,  0.,  0.]),
 array([0.        , 0.        , 1.57079633]),
 array([10.,  0.,  0.]),
 array([10.,  0.,  0.]),
 array([0.        , 0.        , 1.57079633]),
 array([10.,  0.,  0.]),
 array([10.,  0.,  0.]),
 array([0.        , 0.        , 1.57079633])]